In [ ]:
import numpy as np 
import pandas as pd 
import folium
import geopandas as gpd
import shapely
import shapely.geometry as geom

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/california-housing-prices/housing.csv')
df_z = pd.read_csv('/kaggle/input/us-zip-codes-with-lat-and-long/zip_lat_long.csv')

In [ ]:
df.info()

In [ ]:
df_

In [ ]:
gdf= gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df['longitude'],df['latitude']))

In [ ]:
gdf.crs= {'init': 'epsg:4326'}

In [ ]:
gdf.plot()

In [ ]:
gdf_zip=gpd.read_file('/kaggle/input/zipcode-shape-files/tl_2015_us_zcta510.shp')

In [ ]:
gdf_zip.head()

In [ ]:
gdf_zip.drop(columns=['CLASSFP10','MTFCC10','FUNCSTAT10','ALAND10','AWATER10','INTPTLAT10','INTPTLON10'],inplace=True)

In [ ]:
gdf_zip.head()

In [ ]:
gdf_zip.rename(columns={'ZCTA5CE10':'Zipcode'},inplace=True)

In [ ]:
gdf_zip.info()

In [ ]:
gdf_zip['Zipcode'].apply(lambda x:x.zfill(5))

In [ ]:
gdf_zip['Zipcode']=gdf_zip['Zipcode'].apply(lambda x:x.zfill(5))

In [ ]:
gdf_zip.sort_values('Zipcode',inplace=True)

In [ ]:
gdf_zip.head()

In [ ]:
gdf_zip.crs= {'init': 'epsg:4326'}

In [ ]:
gdf.shape

In [ ]:
gdf_ca=gpd.sjoin(gdf_zip,gdf,op="intersects")

In [ ]:
gdf_ca.shape

In [ ]:
gdf_ca.head()

In [ ]:
gdf_ca.reset_index(drop=True,inplace=True)

In [ ]:
gdf_ca.drop(columns=['GEOID10','index_right'],inplace=True)

In [ ]:
gdf_ca.info()

In [ ]:
gdf_ca.to_file("geo_ca_with_housing_price.geojson", driver='GeoJSON')

Plot the data

In [ ]:
gdf=gpd.read_file('/kaggle/working/geo_ca_with_housing_price.geojson',driver='GEOJSON')

In [ ]:
gdf.head(3)

In [ ]:
gdf_ca_hma=gpd.GeoDataFrame(gdf.groupby('Zipcode').agg({'housing_median_age':'mean','geometry':'first'}).reset_index())
gdf_ca_hma.crs= {'init': 'epsg:4326'}

In [ ]:
gdf_ca_hma.head(100)

In [ ]:
gdf_ca_hma.info()

In [ ]:
m = folium.Map(location=[36.778259, -119.417931], zoom_start=3)
folium.Choropleth(
    geo_data=gdf_ca_hma,
    name='choropleth',
    data=gdf_ca_hma,
    columns=['Zipcode', 'housing_median_age'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='housing median age'
).add_to(m)

folium.LayerControl().add_to(m)

m